# Data Collection & Wrangling


**Goal:** Build a stratified 5000 movie list, fetch OMDb movies metadata, fill missing BoxOffice and output clean dataframe.




## 1. Setup

In [1]:
import os
import json
import pandas as pd
import math
import re
import warnings

# ignores annoying warnings
warnings.filterwarnings(
    "ignore",
    message=".*apply operated on the grouping columns.*",
    category=DeprecationWarning
)

base_dir = os.getcwd()
# obdb paths
json_path = os.path.join(base_dir, "..", "data_collection", "omdb_data.json")
csv_output_path = os.path.join(base_dir, "..", "data_collection", "omdb_cleaned.csv")
# kaggle metadata path
metadata_path = os.path.join(base_dir, "..", "data_collection", "Kaggle The Movies Dataset", "movies_metadata.csv")
# master movie list parth
movie_list_path = os.path.join(base_dir, "..", "data_collection", "movie_list.txt")

## 2. Building Master Movie List

description of this section, our idea and an advantage of this approach

### 2.1 Load and preprocess medadata

what we do

In [2]:
df = pd.read_csv(metadata_path, low_memory=False)

df = df.dropna(subset=["budget", "release_date"])
df["budget"] = pd.to_numeric(df["budget"], errors="coerce")
df["year"]   = pd.to_datetime(df["release_date"], errors="coerce").dt.year
df = df[df["year"].between(1974, 2024)]  # 50 years span

### 2.2 Define budget tiers and draw sample

why we make tiers



In [3]:
df["decade"]  = (df["year"] // 10) * 10
df["budget_m"] = df["budget"] / 1000000

budget_bins  = [0, 5, 20, 100, df["budget_m"].max()+1]
budget_labels= ["<5M", "5–20M", "20–100M", ">100M"]
df["budget_tier"] = pd.cut(df["budget_m"], bins=budget_bins, labels=budget_labels)

n_decades = df["decade"].nunique()
n_tiers   = df["budget_tier"].nunique()
total_cells = n_decades * n_tiers

N_PER_CELL = math.ceil(5000 / total_cells)
print(f"{n_decades=} × {n_tiers=} = {total_cells} cells")
print(f"If we want to reach ~5000 movies, then N_PER_CELL = {N_PER_CELL}")

n_decades=6 × n_tiers=4 = 24 cells
If we want to reach ~5000 movies, then N_PER_CELL = 209


what is stratified sampling

how this is gonna help us?

In [4]:
sampled = (
    df
    .groupby(["decade", "budget_tier"], observed=True, group_keys=False)
    .apply(
        lambda grp: grp.sample(min(len(grp), N_PER_CELL), random_state=42),
        include_groups=True)
)
print(f"Total sampled: {len(sampled)}")
print(f"Sampled {len(sampled)} movies across "
      f"{sampled['decade'].nunique()} decades × "
      f"{sampled['budget_tier'].nunique()} budget tiers")

Total sampled: 2971
Sampled 2971 movies across 5 decades × 4 budget tiers


we end up with 2971 movies but we want more!

so now we are going to randomly choose the rest from the original Kaggle dataset to reach 5k

In [5]:
target = 5000
current = len(sampled)
if current < target:
    residual = target - current
    remaining = df.loc[~df.index.isin(sampled.index)] # all except already chosen
    extra = remaining.sample(residual, random_state=42)    # randomly xhoosing
    sampled = pd.concat([sampled, extra]) #merge
print(f"Final stratified + random sample: {len(sampled)} movies")

Final stratified + random sample: 5000 movies


### 2.3 Final master movie list
done

In [6]:
sampled["title"].to_csv(movie_list_path, index=False, header=False)
print(f"Master list of {len(sampled)} titles is saved to {movie_list_path}")

Master list of 5000 titles is saved to c:\PythonProjects\Movie-Team\jupyter notebooks\..\data_collection\movie_list.txt


## 3. OMDb Data Fetching


Refer to `omdb_fetch.py` script

In [7]:
# python omdb_fetch.py

## 4. Inspect Raw JSON

text

json to dataframe

In [8]:
# into dataframe
with open(json_path, "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.head(5)

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,totalSeasons
0,Good Guys Wear Black,1978,PG,01 Jun 1978,95 min,Action,Ted Post,"Bruce Cohn, Mark Medoff, Joseph Fraley","Chuck Norris, Anne Archer, James Franciscus",An ex-US Army commando must find the reason wh...,...,5.1,"3,865",tt0079227,movie,N/A,N/A,N/A,N/A,True,NaN
1,Grizzly,1976,PG,21 May 1976,91 min,"Adventure, Horror, Thriller",William Girdler,"Harvey Flaxman, David Sheldon, Andrew Prine","Christopher George, Andrew Prine, Richard Jaeckel",An eighteen-foot-tall grizzly bear terrorizes ...,...,5.2,"6,166",tt0074593,movie,N/A,N/A,N/A,N/A,True,NaN
2,Midnight Express,1978,R,27 Oct 1978,121 min,"Biography, Crime, Drama",Alan Parker,"Oliver Stone, Billy Hayes, William Hoffer","Brad Davis, Irene Miracle, Bo Hopkins","Billy Hayes, an American college student, is c...",...,7.5,"90,240",tt0077928,movie,N/A,"$35,000,000",N/A,N/A,True,NaN
3,Death Wish,2018,R,02 Mar 2018,107 min,"Action, Crime, Drama",Eli Roth,"Joe Carnahan, Brian Garfield, Wendell Mayes","Bruce Willis, Vincent D'Onofrio, Elisabeth Shue",Dr. Paul Kersey is an experienced trauma surge...,...,6.3,"78,823",tt1137450,movie,N/A,"$34,017,028",N/A,N/A,True,NaN
4,Capone,2020,R,12 May 2020,103 min,"Biography, Crime, Drama",Josh Trank,Josh Trank,"Tom Hardy, Linda Cardellini, Matt Dillon","After a 10 year stint, former bootlegger and n...",...,4.7,"23,898",tt6199572,movie,N/A,N/A,N/A,N/A,True,NaN


## 5. Data Cleaning

text

### 5.1 Selecting relevant columns

In [9]:
columns_to_keep = [
    "Title", "imdbID", "Year", "Genre", "Director", "Actors", "Language",
    "Country", "Runtime", "BoxOffice", "imdbRating", "imdbVotes", "Awards", "Type"
]

df = df[columns_to_keep]
df.head(3)

,Title,imdbID,Year,Genre,Director,Actors,Language,Country,Runtime,BoxOffice,imdbRating,imdbVotes,Awards,Type
0,Good Guys Wear Black,tt0079227,1978,Action,Ted Post,"Chuck Norris, Anne Archer, James Franciscus",English,United States,95 min,N/A,5.1,"3,865",N/A,movie
1,Grizzly,tt0074593,1976,"Adventure, Horror, Thriller",William Girdler,"Christopher George, Andrew Prine, Richard Jaeckel",English,United States,91 min,N/A,5.2,"6,166",N/A,movie
2,Midnight Express,tt0077928,1978,"Biography, Crime, Drama",Alan Parker,"Brad Davis, Irene Miracle, Bo Hopkins","English, Turkish, French","United Kingdom, United States",121 min,"$35,000,000",7.5,"90,240",Won 2 Oscars. 17 wins & 14 nominations total,movie


### 5.2 Cleaning numeric fields + budget columb

In [10]:
#BoxOffice to integer
df["BoxOffice"] = df["BoxOffice"].replace("N/A", pd.NA)
df["BoxOffice"] = df["BoxOffice"].dropna().apply(
    lambda x: int(re.sub(r"[\$,]", "", x)) if isinstance(x, str) else pd.NA
)

#imdbRating to float
df["imdbRating"] = pd.to_numeric(df["imdbRating"], errors="coerce")

#imdbVotes to integer
df["imdbVotes"] = df["imdbVotes"].replace("N/A", pd.NA)
df["imdbVotes"] = df["imdbVotes"].dropna().apply(
    lambda x: int(x.replace(",", "")) if isinstance(x, str) else pd.NA
)

# here we wanna merge budget from the original Kaggle metadata
meta_budget = pd.read_csv(
    metadata_path,
    usecols=["imdb_id", "budget"],
    low_memory=False
)
meta_budget["budget"] = pd.to_numeric(meta_budget["budget"], errors="coerce")
meta_budget = meta_budget.rename(columns={"imdb_id": "imdbID"})

# we do the left-join
df = df.merge(
    meta_budget[["imdbID", "budget"]],
    on="imdbID",
    how="left"
)

# saving cleaned data to csv
df.to_csv(csv_output_path, index=False)

### 5.3 Removing duplicates

In [11]:
before = len(df)
df = df.drop_duplicates(subset=["imdbID"], keep="first").reset_index(drop=True)
after = len(df)
print(f"Dropped {before - after} duplicate records; {after} unique movies remain.")

Dropped 80 duplicate records; 4688 unique movies remain.


### 5.4 Zeros to NaN in BoxOffice

In [12]:
df["BoxOffice_was_missing"] = (df["BoxOffice"] == 0)

df.loc[df["BoxOffice"] == 0, "BoxOffice"] = pd.NA

zero_count = df["BoxOffice_was_missing"].sum()
nan_count  = df["BoxOffice"].isna().sum()

## 6. Box Office Missing Values

After cleaning, we check how many movies do not contain valid box office revenue data.

In [13]:
boxoffice_miss = df[df["BoxOffice"].isna()]
print(f"{len(boxoffice_miss)} out of {len(df)} movies are missing BoxOffice data.")

1902 out of 4688 movies are missing BoxOffice data.


### 6.1 Merging IMDb and TMDb IDs

1 load raw json and locate imdbID

2 join with links.csv from Kaggle to get tmdb 



TMDB
API KEY - 1d779d5d4246f2809fc00d7729449f09
API Read Access Token - eyJhbGciOiJIUzI1NiJ9



In [14]:
# I load json as dataframe
with open(json_path, "r", encoding="utf-8") as f:
    omdb_raw = json.load(f)
omdb_raw_df = pd.DataFrame(omdb_raw)

# links.csv dataframe from Kaggle(imdbId keyword)
links_path = os.path.join(base_dir, "..", "data_collection", "Kaggle The Movies Dataset", "links.csv")
links_df = pd.read_csv(links_path, usecols=["imdbId", "tmdbId"])

missing = df[df["BoxOffice"].isna()].copy() # titles still missing Boxofice

# taking out numbers from imdbID field tt0123456 to 123456
missing["imdbId_numeric"] = (missing["imdbID"]
    .str.replace("^tt", "", regex=True)
    .astype("Int64")
)

missing = missing.merge( # and now merge on that numeric field
    links_df, left_on="imdbId_numeric", right_on="imdbId", how="left")

### 6.2 Save enhanced missing list

a

In [15]:
enhanced_cols = ["Title","imdbID","tmdbId"]
enhanced_path = os.path.join(
    base_dir, "..", "data_collection", "missing_boxoffice_enhanced.csv"
)
missing[enhanced_cols].to_csv(enhanced_path, index=False)
print(f"Enhanced missing list ({len(missing)}) was saved to {enhanced_path}")

Enhanced missing list (1902) was saved to c:\PythonProjects\Movie-Team\jupyter notebooks\..\data_collection\missing_boxoffice_enhanced.csv


### 6.3 TMDb API call

There we call TMDb to fill in `revenue` where boxoffice is EMPTY

refer to `tmdb_fetch.py` script to replicate

1657 out of 1919 missing revenue movies have been successfully fetched!

In [16]:
# python tmdb_fetch.py

### 6.4 Merging TMDb revenues with `df`


text

In [17]:
tmdb_rev = pd.read_csv(
    os.path.join(base_dir, "..", "data_collection", "tmdb_revenues.csv")
)

#merging into df by title
df = df.merge(tmdb_rev[["Title","BoxOffice"]],
    on="Title", how="left", suffixes=("", "_tmdb")
)

#wherever BoxOffice is null = fill from the tmdb column
df["BoxOffice"] = df["BoxOffice"].fillna(df["BoxOffice_tmdb"])
df = df.drop(columns=["BoxOffice_tmdb"])

still_missing = df["BoxOffice"].isna().sum()
total_movies  = len(df)
print(f"{still_missing} out of {total_movies} movies still have no BoxOffice after TMDb API calling.")

256 out of 4688 movies still have no BoxOffice after TMDb API calling.


### 6.5 Building the scraper list

here we are going to find still missing revenue 262 movies

THose 262 titles fall into 2 possible categories:

- no tmdb ID at all (hence we couldn't call TMDb API)
- title has tmdb id but API returned zero because it may not track this title

That being said, we will use "The-Numbers" in order to scrape them again!


In [18]:
#still missing titles
still_missing_titles = df[df["BoxOffice"].isna()]["Title"]

# filtering missing titles DataFrame (the one with tmdbId) by those titles
fallback = missing[missing["Title"].isin(still_missing_titles)][
    ["Title","imdbID","tmdbId"]
]

fallback_path = os.path.join(
    base_dir, "..", "data_collection", "fallback_boxoffice.csv"
)
fallback.to_csv(fallback_path, index=False)

print(f"{len(fallback)} titles to scrape are saved to {fallback_path}")

256 titles to scrape are saved to c:\PythonProjects\Movie-Team\jupyter notebooks\..\data_collection\fallback_boxoffice.csv


### 6.6 Scraping on The-Numbers

To replicate, refer to `thenumbers_fetch.py`


In [19]:
# python thenumbers_fetch.py

Unfortunately, all of 262 titles couldn't get the box office information. In such case we replace their values onto NaN and keep them for further exploratory work.

### 6.7 "Zeros" to NaN in boxoffice


In [20]:
df["BoxOffice_was_missing"] = (df["BoxOffice"] == 0)

# zeros to NaN
df.loc[df["BoxOffice"] == 0, "BoxOffice"] = pd.NA

zero_count = (df["BoxOffice_was_missing"]).sum()
nan_count  = df["BoxOffice"].isna().sum()
print(f"{zero_count} movies had zero BoxOffice and are now NaN (total NaNs: {nan_count})")

1411 movies had zero BoxOffice and are now NaN (total NaNs: 1667)


## 7. Final Data Overview

- missing BoxOffice values are converted to NaN
- Key numeric summaries (e.g. ratings, votes..)
- ...

In [21]:
zero_count     = (df["BoxOffice"] == 0).sum()
na_count       = df["BoxOffice"].isna().sum()
positive_count = (df["BoxOffice"] > 0).sum()
total          = len(df)

print(f"BoxOffice == 0:    {zero_count} movies")
print(f"BoxOffice is NaN:  {na_count} movies")
print(f"BoxOffice > 0:     {positive_count} movies")
print(f"Total checked:     {total} movies")


BoxOffice == 0:    0 movies
BoxOffice is NaN:  1667 movies
BoxOffice > 0:     3021 movies
Total checked:     4688 movies


In [22]:
summary = df[["BoxOffice", "imdbRating", "imdbVotes", "Runtime"]].describe().T

numeric_cols = summary.index.tolist()
missing_counts = df[numeric_cols].isna().sum()
summary["missing"] = missing_counts

display(summary)

,count,mean,std,min,25%,50%,75%,max,missing
BoxOffice,3021.0,4.368908e+07,7.371442e+07,13.0,1436243.0,15379253.0,51293931.00,785221649.0,1667
imdbRating,4630.0,6.307732e+00,1.118031e+00,1.3,5.7,6.4,7.10,9.5,58
imdbVotes,4646.0,8.616201e+04,2.056882e+05,5.0,2129.5,11969.5,70352.75,2997696.0,42


In [23]:
print("Top 5 movies by BoxOffice:")
display(df.nlargest(5, "BoxOffice")[["Title","Year","BoxOffice"]])

print("\n5 lowest movies by BoxOffice:")
display(df.nsmallest(500, "BoxOffice")[["Title","Year","BoxOffice"]])

Top 5 movies by BoxOffice:


,Title,Year,BoxOffice
2014,Avatar,2009,785221649.0
971,Titanic,1997,674354882.0
2736,Jurassic World,2015,653406625.0
2842,The Avengers,2012,623357910.0
1982,The Dark Knight,2008,534987076.0



5 lowest movies by BoxOffice:


,Title,Year,BoxOffice
2153,The King of Pigs,2011,13.0
3506,Windwalker,1980,18.0
889,The Immortals,1995,83.0
1567,The Pornographer,2001,126.0
2422,Playback,2012,264.0
...,...,...,...
3188,Koch,2012,342941.0
2975,American Adobo,2001,344992.0
325,Prison,1987,345704.0
1047,Two Evil Eyes,1990,349618.0


In [24]:
df["Genre_list"] = df["Genre"].str.split(", ")

top_genres = (
    df.explode("Genre_list")["Genre_list"]
      .value_counts()
      .head(10)
)
print("Table of Frequencies: top 10 genres")
display(top_genres)

print("\nTable of Frequencies: top 10 directors by № of movies")
display(df["Director"].value_counts().head(10))

Table of Frequencies: top 10 genres


Genre_list
Drama        2310
Comedy       1588
Action       1109
Adventure     828
Crime         796
Thriller      691
Romance       671
Horror        564
Mystery       369
Fantasy       357
Name: count, dtype: int64


Table of Frequencies: top 10 directors by № of movies


Director
N/A                 119
Steven Spielberg     17
Ridley Scott         16
Martin Scorsese      13
Woody Allen          13
John Carpenter       13
Michael Bay          12
Tim Burton           12
Ron Howard           11
David Cronenberg     11
Name: count, dtype: int64